In [48]:
include("MiniCollectiveSpins.jl")
using PyPlot
using Statistics
using JLD2
import PhysicalConstants.CODATA2018: c_0
using Unitful
using ProgressMeter
using NonlinearSolve
using SteadyStateDiffEq
using Libdl
using Symbolics
using OrdinaryDiffEq

In [49]:
""" Prepare the initial vector u0 """
function u0_CFunction(phi_array, theta_array, op_list)
    u0 = ones(ComplexF64, length(op_list))
    for i in 1:length(op_list)
        if length(op_list[i]) == 1
            j = Int(op_list[i][1] % 10^floor(log10(abs(op_list[i][1]))-1)) # Atom nbr
            if parse(Int, string(op_list[i][1])[1:2]) == 22
                u0[i] = cos(theta_array[j]/2)^2
            elseif parse(Int, string(op_list[i][1])[1:2]) == 21
                u0[i] = cos(theta_array[j]/2)*exp(1im*phi_array[j])*sin(theta_array[j]/2)
            else
                println(op_list[i][1])
            end
        end

        if length(op_list[i]) == 2
            for op in op_list[i]
                j = Int(op % 10^floor(log10(abs(op))-1)) # Atom nbr
                if parse(Int, string(op)[1:2]) == 22
                    u0[i] *= cos(theta_array[j]/2)^2
                elseif parse(Int, string(op)[1:2]) == 21
                    u0[i] *= cos(theta_array[j]/2)*exp(1im*phi_array[j])*sin(theta_array[j]/2)
                elseif parse(Int, string(op)[1:2]) == 12
                    u0[i] *= cos(theta_array[j]/2)*exp(-1im*phi_array[j])*sin(theta_array[j]/2)
                else
                    println(op)
                end
            end
        end
    end
    return u0
end

""" Function loading the block subfunction when a lot of equations are involved """
function load_f(fname::String, libpath::String)
	lib = Libdl.dlopen(libpath)
	fptr = Libdl.dlsym(lib, fname)
	return (du, u, params) -> ccall(fptr, Cvoid, (Ptr{ComplexF64}, Ptr{ComplexF64}, Ptr{ComplexF64}), du, u, params)
end

load_f

### Define the system

In [50]:
# Nbr of particles
N_list = [2:2:10;];
r = 100 # Nbr of repetitions

# Normalisation parameters
λ = 421e-9
γ = 32.7e6 # In Hz

# Physical values
ω0 = (2π*ustrip(c_0)/λ)
ωl = ω0
kl = [ustrip(c_0)/ωl, 0, 0] # Laser along x
Ω_RO = 1e7 # Taken from Barbut arXiv:2412.02541v1

# Size of the box
Lx, Ly, Lz = [1, 1, 1] * 1e-6

# Normalization
ω0 = ω0 / γ
ωl = ωl / γ
kl = kl * λ
Ω_RO = Ω_RO / γ

# Quantization axis along z
e = [0, 0, 1.];

In [51]:
# Prepare the wrapper
const N_FUNCS = length(N_list)  # Total function nbr
const functions = Vector{Function}(undef, N_FUNCS)

for (i, N) in enumerate(N_list)
    libpath ="libs/liballfuncs_$N.dll"
    functions[i] = load_f("diffeqf", libpath)
end

# Compute stationary state for each atom number

In [52]:
# Create the directories
if !isdir("r0")
    mkdir("r0")
end
# Create the directories
if !isdir("Images_distribution")
    mkdir("Images_distribution")
end

In [ ]:
popup_SS_N = []

for (i, N) in enumerate(N_list)
    print("N = $N")
    popup_SS = []
    @showprogress for j = 1:r
        r0 = [[rand(Float64)*Lx, rand(Float64)*Ly, rand(Float64)*Lz] for k in 1:N]
        # Save the atoms position for comparison with QuantumOptics
        @save "r0/r0_N_$(N)_r_$j.jdl2" r0

        # Normalize the position vectors
        r0 = r0 / λ

        # Plot distribution
        close("all")
        fig = plt.figure()
        ax = fig.add_subplot(projection="3d")
        ax.scatter([r[1] for r in r0], [r[2] for r in r0], [r[3] for r in r0])
        ax.set_xlabel(L"x/$\lambda$"); ax.set_ylabel(L"y/$\lambda$"); ax.set_zlabel(L"z/$\lambda$")
        ax.set_xlim(0, Lx/λ), ax.set_ylim(0, Ly/λ), ax.set_zlim(0, Lz/λ)
        plt.savefig("Images_distribution/Images_distribution_N_$(N)_r_$j")

        # Compute the Ω and Γ matrices of the electric dipole-dipole interaction using CollectiveSpins
        system = SpinCollection(r0, e, gammas=1.)
        Ω_CS = OmegaMatrix(system)
        Γ_CS = GammaMatrix(system)
        Γij_ = [Γ_CS[i, j] for i = 1:N for j=1:N]
        Ωij_ = [Ω_CS[i, j] for i = 1:N for j=1:N if i≠j]
        exp_RO_ = [exp(1im*r0[i]'kl) for i = N:-1:1] # We go in the decreasing direction to avoid exp_RO(10) being replace by exp_RO(1)0
        conj_exp_RO_ = [exp(-1im*r0[i]'kl) for i = N:-1:1]
        p0 = ComplexF64.([Γij_; Ωij_; exp_RO_; conj_exp_RO_; Ω_RO/2])

        # Load the functions
        fsolve(du, u, p, t) = functions[i](du, u, p0)

        ## Compute stationnary state ## 
        # Prepare the guess for the steady state
        @load "op_list/op_list_$N.jdl2" op_list
        phi_array_f, theta_array_f = zeros(N), ones(N)*3π/4
        uf = u0_CFunction(phi_array_f, theta_array_f, op_list)

        # Compute stationnary state
        prob_ss = SteadyStateProblem(fsolve, uf)
        sol_ss = solve(prob_ss, SSRootfind())
        stationnary_state = sum(real(sol_ss[1:N]))
        push!(popup_SS, stationnary_state)
    end
    push!(popup_SS_N, popup_SS)
end

nothing

N = 2

Progress: 100%|█████████████████████████████████████████| Time: 0:00:28


N = 4

Progress: 100%|█████████████████████████████████████████| Time: 0:00:31


N = 6

Progress: 100%|█████████████████████████████████████████| Time: 0:01:18


N = 8

Progress: 100%|█████████████████████████████████████████| Time: 0:16:57


N = 10

Progress:  34%|██████████████                           |  ETA: 2:11:59

# Plots

In [ ]:
close("all")
fig = figure()
errorbar(N_list, [mean(popup_SS) for popup_SS in popup_SS_N], yerr=[std(popup_SS) for popup_SS in popup_SS_N])
xlabel(L"N")
ylabel(L"$\langle  n_{\uparrow}^\infty \rangle$")
suptitle("N = $N, r = $r, Starting from "*L"$|\downarrow \downarrow \rangle $")
pygui(false); gcf()
# pygui(true); show()